In [25]:
# Imports
import numpy as np
import pandas as pd
import polars as pl
import pulp
from tqdm.auto import tqdm

from warnings import filterwarnings

filterwarnings("ignore")

# Load master data
vehicles_df = pd.read_csv("../data/Master_data/vehicle_master_data.csv")
distances_df = pd.read_csv("../data/Master_data/distance_master_data_2.csv")

In [26]:
# Read input
input_df = pd.read_csv("../data/Input_data/Input5.csv")

In [27]:
# Createing decision variable column in the dataframe
df = distances_df.join(vehicles_df, how="cross")
df["decisions"] = df["Source"] + ", " + df["Destination"] + ", " + df["Transportation modes"] + ", " + df["Vehicle_type"]

In [28]:
# Drop mode distance that are not compatible to the specific vehicle
df.drop(
    df[(df["Air Distance (km)"].isna()) & (df["Transportation modes"] == "Air")].index,
    axis=0,
    inplace=True,
)
df.drop(
    df[(df["Rail Distance (km)"].isna()) & (df["Transportation modes"] == "Rail")].index,
    axis=0,
    inplace=True,
)
df.drop(
    df[(df["Road Distance (km)"].isna()) & (df["Transportation modes"] == "Road")].index,
    axis=0,
    inplace=True,
)
df.reset_index(inplace=True, drop=True)

In [29]:
# Pre-calculate time, cost and obj function value
time_list = []
cost_list = []
obj1_list = []

for i in range(len(df)):
    if df["Transportation modes"][i] == "Air":
        time_list.append(df["Air Distance (km)"][i] / df["Avg Speed (km/hr)"][i])
        cost_list.append(df["Air Distance (km)"][i] * df["Cost per km"][i])
        obj1_list.append(df["Air Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)

    elif df["Transportation modes"][i] == "Road":
        time_list.append(df["Road Distance (km)"][i] / df["Avg Speed (km/hr)"][i])
        cost_list.append(df["Road Distance (km)"][i] * df["Cost per km"][i])
        obj1_list.append(df["Road Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)

    else:
        time_list.append(df["Rail Distance (km)"][i] / df["Avg Speed (km/hr)"][i])
        cost_list.append(df["Rail Distance (km)"][i] * df["Cost per km"][i])
        obj1_list.append(df["Rail Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)


df["Time (hrs)"] = time_list
df["Cost"] = cost_list
df["obj1"] = obj1_list

In [30]:
# Filtering the model data according to provided input data
i1 = df.set_index(list(df[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index

df0 = df[i1.isin(i2)]
df0.reset_index(drop=True, inplace=True)

In [31]:
# pandas -> polars (for faster performance)
df1 = pl.from_pandas(df0)
df1

Source,Destination,Air Distance (km),Rail Distance (km),Road Distance (km),Transportation modes,Vehicle_type,Capacity (metric tons),Co2e (g/km),Avg Speed (km/hr),Cost per km,Availability,Minimum Quantity Allowed (MQA),decisions,Time (hrs),Cost,obj1
str,str,f64,f64,f64,str,str,i64,f64,i64,f64,i64,f64,str,f64,f64,f64
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Air""","""CARGO PLANE""",100,551.584454,500,20.0,1,90.0,"""Delhi, Mumbai,…",2.28704,22870.4,630.747855
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Rail""","""GOODS RAIL""",8000,97.565914,50,4.0,1,4000.0,"""Delhi, Mumbai,…",23.8056,4761.12,116.130756
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Rail""","""MIXED RAIL""",5000,107.090591,90,5.5,1,2000.0,"""Delhi, Mumbai,…",13.225333,6546.54,127.467789
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Road""","""HGV DROPSIDE L…",24,156.598994,45,1.3,2,10.0,"""Delhi, Mumbai,…",31.798,1860.183,224.079066
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Road""","""4WD PICK UP""",18,155.605476,42,1.2,2,10.0,"""Delhi, Mumbai,…",34.069286,1717.092,222.657432
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Road""","""VAN""",12,211.365144,70,1.4,2,5.0,"""Delhi, Mumbai,…",20.441571,2003.274,302.444499
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Road""","""PICK UP""",10,209.320194,55,1.34,2,3.0,"""Delhi, Mumbai,…",26.016545,1917.4194,299.518359
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Road""","""HGV (Large)""",36,155.398594,34,1.23,2,20.0,"""Delhi, Mumbai,…",42.085588,1760.0193,222.361401
"""Delhi""","""Mumbai""",1143.52,1190.28,1430.91,"""Road""","""CAR DERIVED VA…",2,187.850299,65,1.41,2,0.5,"""Delhi, Mumbai,…",22.014,2017.5831,268.796871


In [32]:
# master availability mapping
master_availability = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Availability"]))

# master demand mapping
master_demand = {}
for i, j in zip(
    np.array(df[["Source", "Destination"]].drop_duplicates()),
    [
        0,
    ]
    * len(df[["Source", "Destination"]].drop_duplicates()),
):
    master_demand[tuple(i)] = j

# real/input demand mapping
demand = {}
for i, j in zip(np.array(input_df[["From", "To"]]), input_df["Quantity (MT)"]):
    demand[tuple(i)] = j

for k in master_demand.keys():
    if k not in demand.keys():
        demand[k] = 0

In [33]:
%%time

# Result lists choices
vehicle_count_choice_list = []
material_transport_quantity_choice_list = []
co2_emission_choice_list = []
transportation_cost_choice_list = []
transportation_time_choice_list = []

# Result lists trips/return trips
vehicle_count_trip_list = []
material_transport_quantity_trip_list = []
co2_emission_trip_list = []
transportation_cost_trip_list = []
transportation_time_trip_list = []

##########################################################
# ------------------- Model Definition ------------------#
##########################################################
# Create a LP minimization problem
model = pulp.LpProblem("Co2e Optimization", pulp.LpMinimize)

############################################################
# ------------------- Decision Variables ------------------#
############################################################

# choice decision variable -> Bool
choice_decision = pulp.LpVariable.dicts(
    "Choice_Decision_",
    (tuple(df1["decisions"][i].split(", ")) + (j,) for i in range(len(df1)) for j in range(master_availability[df1["decisions"][i].split(", ")[-1]])),
    lowBound=0,
    cat="Binary",
)

# trip/return trip decision variable -> Int
trip_decision = pulp.LpVariable.dicts(
    "Trip_Decision_",
    (tuple(df1["decisions"][i].split(", ")) + (j,) for i in range(len(df1)) for j in range(master_availability[df1["decisions"][i].split(", ")[-1]])),
    lowBound=0,
    cat="Integer",
)

############################################################
# ------------------- Objective Function ------------------#
############################################################

obj_list = []
for s, d, m, v, c in choice_decision.keys():
    obj_list.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["obj1"].item())
    obj_list.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["obj1"].item())

model += pulp.lpSum(obj_list)

#####################################################
# ------------------- Constraints ------------------#
#####################################################

# ========== Demand Constraint ==========
(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for s, d, m, v, c in choice_decision.keys():
    if (s0, d0) == (s, d):
        col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
        col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
    else:
        if demand[(s0, d0)] == 0:
            model += pulp.lpSum(col) == demand[(s0, d0)]
            (s0, d0) = (s, d)
            col = []
            col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
            col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
        elif demand[(s0, d0)] >= 0:
            model += pulp.lpSum(col) >= demand[(s0, d0)]
            (s0, d0) = (s, d)
            col = []
            col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
            col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())

if demand[(s0, d0)] == 0:
    model += pulp.lpSum(col) == demand[(s0, d0)]
elif demand[(s0, d0)] >= 0:
    model += pulp.lpSum(col) >= demand[(s0, d0)]


# ========== Minimum Quantity Allowed Constraint ==========

(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for s, d, m, v, c in choice_decision.keys():
    if (s0, d0) == (s, d):
        col.append(
            choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item()
        )
        col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item())
    else:
        if demand[(s0, d0)] == 0:
            model += pulp.lpSum(col) == demand[(s0, d0)]
            (s0, d0) = (s, d)
            col = []
            col.append(
                choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item()
            )
            col.append(
                trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Minimum Quantity Allowed (MQA)"].item()
            )
        elif demand[(s0, d0)] >= 0:
            model += pulp.lpSum(col) <= demand[(s0, d0)]
            (s0, d0) = (s, d)
            col = []
            col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())
            col.append(trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Capacity (metric tons)"].item())

if demand[(s0, d0)] == 0:
    model += pulp.lpSum(col) == demand[(s0, d0)]
elif demand[(s0, d0)] >= 0:
    model += pulp.lpSum(col) <= demand[(s0, d0)]


# ========== Time Constraint ==========

for s, d, m, v, c in choice_decision.keys():
    if input_df[(input_df["From"] == s) & (input_df["To"] == d)].shape[0]:
        exp = (choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Time (hrs)"].item()) + (
            2.5 * trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Time (hrs)"].item()
        )
        model += pulp.LpConstraint(
            e=pulp.LpAffineExpression(exp),
            sense=pulp.LpConstraintLE,
            rhs=input_df[(input_df["From"] == s) & (input_df["To"] == d)]["Time (hrs)"].item(),
        )


# ========== Cost Constraint ==========

(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for s, d, m, v, c in choice_decision.keys():
    if ((s0, d0) == (s, d)) & input_df[(input_df["From"] == s) & (input_df["To"] == d)].shape[0]:
        col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
        col.append( trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
    elif (s0, d0) != (s, d):
        if input_df[(input_df["From"] == s0) & (input_df["To"] == d0)].shape[0]:
            model += pulp.lpSum(col) <= input_df[(input_df["From"] == s0) & (input_df["To"] == d0)]["Cost"]
            (s0, d0) = (s, d)
            col = []
            col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
            col.append( trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
        else:
            (s0, d0) = (s, d)
            col = []

if input_df[(input_df["From"] == s) & (input_df["To"] == d)].shape[0]:
    model += pulp.lpSum(col) <= input_df[(input_df["From"] == s0) & (input_df["To"] == d0)]["Cost"]

# cost_1=col
# ========== Other Constraints ==========

for s, d, m, v, c in choice_decision.keys():
    model += trip_decision[(s, d, m, v, c)] <= 10000000 * choice_decision[(s, d, m, v, c)]

for s, d, m, v, c in choice_decision.keys():
    if m != "Road":
        model += trip_decision[(s, d, m, v, c)] == 0

###############################################
# ------------------- Solve ------------------#
###############################################

model.solve()
print(pulp.LpStatus[model.status])

#######################################################################
# ------------------- Mappings Necessary for Output ------------------#
#######################################################################

# Extracting necessary output info. from the solver
mt_choice = []
mt_trip = []
for v in model.variables():
    if v.varValue != 0:
        if v.name.split("__(")[0] == "Choice_Decision":
            mt_choice.append(
                v.name.split("__(")[1].rstrip(")").split(",_")
                + [
                    v.varValue,
                ]
            )
        else:
            mt_trip.append(
                v.name.split("__(")[1].rstrip(")").split(",_")
                + [
                    v.varValue,
                ]
            )

mt_choice_output = pd.DataFrame(
    mt_choice,
    columns=[
        "Source",
        "Destination",
        "Transportation mode",
        "Vehicle type",
        "Vehicle number",
        "Count",
    ],
)
mt_choice_output_df = mt_choice_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_choice_output_df["S_D_M"] = (
    mt_choice_output_df["Source"] + "_" + mt_choice_output_df["Destination"] + "_" + mt_choice_output_df["Transportation mode"]
)

mt_trip_output = pd.DataFrame(
    mt_trip,
    columns=[
        "Source",
        "Destination",
        "Transportation mode",
        "Vehicle type",
        "Vehicle number",
        "Count",
    ],
)
mt_trip_output_df = mt_trip_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_trip_output_df["S_D_M"] = mt_trip_output_df["Source"] + "_" + mt_trip_output_df["Destination"] + "_" + mt_trip_output_df["Transportation mode"]
mt_trip_output_df["Vehicle type with number"] = mt_trip_output_df["Vehicle type"] + "_" + mt_trip_output_df["Vehicle number"]

# Apply all mapping to the output
dist_list = []
vals = []
for c in distances_df.columns[2:]:
    for i in range(len(distances_df)):
        dist_list.append(distances_df["Source"][i] + "_" + distances_df["Destination"][i] + "_" + c.split(" ")[0])
        vals.append(distances_df[c][i])

distance_mappings = dict(zip(dist_list, vals))
quantity_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Capacity (metric tons)"]))
emission_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Co2e (g/km)"]))
cost_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Cost per km"]))
time_mappings = dict(
    zip(
        (df["Source"] + "_" + df["Destination"] + "_" + df["Transportation modes"] + "_" + df["Vehicle_type"].str.replace(" ", "_")),
        df["Time (hrs)"],
    )
)

# Get output data ready before putting into presentable format
mt_choice_output_df["Loaded quantity"] = mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(quantity_mappings)
mt_choice_output_df["Distance"] = mt_choice_output_df["S_D_M"].map(distance_mappings)
mt_choice_output_df["Emission"] = (2 * 
    (mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(emission_mappings)) * mt_choice_output_df["Distance"] / 1000
)
mt_choice_output_df["Transportation cost"] = (mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(cost_mappings)) * mt_choice_output_df[
    "Distance"
]
mt_choice_output_df["Time (hrs)"] = (mt_choice_output_df["S_D_M"] + "_" + mt_choice_output_df["Vehicle type"]).map(time_mappings)

mt_trip_output_df["Loaded quantity"] = mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(quantity_mappings)
mt_trip_output_df["Distance"] = mt_trip_output_df["S_D_M"].map(distance_mappings)
mt_trip_output_df["Emission"] = (2 * 
    (mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(emission_mappings)) * mt_trip_output_df["Distance"] / 1000
)
mt_trip_output_df["Transportation cost"] = (mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(cost_mappings)) * mt_trip_output_df[
    "Distance"
]
mt_trip_output_df["Time (hrs)"] = (mt_trip_output_df["S_D_M"] + "_" + mt_trip_output_df["Vehicle type"]).map(time_mappings)
mt_trip_output_df["Time (hrs)"] *= mt_trip_output_df["Count"]

###########################################################
# ------------------- Generating Output ------------------#
###########################################################

# Pivot tables for more presentable data

vehicle_count_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Count",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
vehicle_count_choice["Total Vehicles in Use"] = vehicle_count_choice.iloc[:, 2:].sum(axis=1)

material_transport_quantity_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Loaded quantity",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
material_transport_quantity_choice["Total Quantity Transported"] = material_transport_quantity_choice.iloc[:, 2:].sum(axis=1)

co2_emission_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Emission",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
co2_emission_choice["Total co2 Emission (kg)"] = co2_emission_choice.iloc[:, 2:].sum(axis=1)

transportation_cost_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Transportation cost",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
transportation_cost_choice["Total transportation cost"] = transportation_cost_choice.iloc[:, 2:].sum(axis=1)

transportation_time_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Time (hrs)",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="max",
).reset_index()
transportation_time_choice["Total Time (hrs)"] = transportation_time_choice.iloc[:, 2:].sum(axis=1)

###########################

vehicle_count_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Count",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
vehicle_count_trip["Total Vehicles in Use"] = vehicle_count_trip.iloc[:, 2:].sum(axis=1)

material_transport_quantity_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Loaded quantity",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
material_transport_quantity_trip["Total Quantity Transported"] = material_transport_quantity_trip.iloc[:, 2:].sum(axis=1)

co2_emission_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Emission",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
co2_emission_trip["Total co2 Emission (kg)"] = co2_emission_trip.iloc[:, 2:].sum(axis=1)

transportation_cost_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Transportation cost",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
transportation_cost_trip["Total transportation cost"] = transportation_cost_trip.iloc[:, 2:].sum(axis=1)

transportation_time_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Time (hrs)",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="max",
).reset_index()
transportation_time_trip[" Total Time (hrs)"] = transportation_time_trip.iloc[:, 2:].sum(axis=1)

# Some cleaning

vehicle_count_choice.fillna(0, inplace=True)
material_transport_quantity_choice.fillna(0, inplace=True)
co2_emission_choice.fillna(0, inplace=True)
transportation_cost_choice.fillna(0, inplace=True)
transportation_time_choice.fillna(0, inplace=True)

vehicle_count_choice_list.append(vehicle_count_choice)
material_transport_quantity_choice_list.append(material_transport_quantity_choice)
co2_emission_choice_list.append(co2_emission_choice)
transportation_cost_choice_list.append(transportation_cost_choice)
transportation_time_choice_list.append(transportation_time_choice)

vehicle_count_trip.fillna(0, inplace=True)
material_transport_quantity_trip.fillna(0, inplace=True)
co2_emission_trip.fillna(0, inplace=True)
transportation_cost_trip.fillna(0, inplace=True)
transportation_time_trip.fillna(0, inplace=True)

vehicle_count_trip_list.append(vehicle_count_trip)
material_transport_quantity_trip_list.append(material_transport_quantity_trip)
co2_emission_trip_list.append(co2_emission_trip)
transportation_cost_trip_list.append(transportation_cost_trip)
transportation_time_trip_list.append(transportation_time_trip)

Optimal
CPU times: total: 1.23 s
Wall time: 617 ms


In [34]:
################################################################################
# ------------------- Generating separate output excel files ------------------#
################################################################################

with pd.ExcelWriter("../data/Output_data/v4/ans1/Vehicle_Count.xlsx") as writer1:
    vehicle_count_choice_list[0].to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    vehicle_count_trip_list[0].to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=vehicle_count_choice_list[0].shape[1] + 2)

with pd.ExcelWriter("../data/Output_data/v4/ans1/Material_Transport_Quantity.xlsx") as writer2:
    material_transport_quantity_choice_list[0].to_excel(writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    material_transport_quantity_trip_list[0].to_excel(
        writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=material_transport_quantity_choice_list[0].shape[1] + 2
    )

with pd.ExcelWriter("../data/Output_data/v4/ans1/CO2_Emission.xlsx") as writer3:
    co2_emission_choice_list[0].to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    co2_emission_trip_list[0].to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=co2_emission_choice_list[0].shape[1] + 2)

with pd.ExcelWriter("../data/Output_data/v4/ans1/Transportation_Cost.xlsx") as writer4:
    transportation_cost_choice_list[0].to_excel(writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    transportation_cost_trip_list[0].to_excel(
        writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_cost_choice_list[0].shape[1] + 2
    )

with pd.ExcelWriter("../data/Output_data/v4/ans1/Transportation_Time.xlsx") as writer5:
    transportation_time_choice_list[0].to_excel(writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    transportation_time_trip_list[0].to_excel(
        writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_time_choice_list[0].shape[1] + 2
    )

In [35]:
##################################################################
# ------------------- Final Summary and Result ------------------#
##################################################################

output_large_merged = mt_choice_output_df.merge(mt_trip_output_df[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count', 'Time (hrs)']], on=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number'], how="left").fillna(-1)
output_large_merged['Count_y']+=1
output_large_merged['Count_y'] = output_large_merged[['Count_x', 'Count_y']].max(axis=1)
output_large_merged['Time (hrs)'] = output_large_merged[['Time (hrs)_x', 'Time (hrs)_y']].max(axis=1)
summary_large = output_large_merged[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count_y', 'Loaded quantity', 'Emission', 'Transportation cost', 'Time (hrs)']]
summary_large.rename({'Count_y': 'Number of trips', 'Emission': 'Total CO2 Emission', 'Transportation cost': 'Total Transportation Cost', 'Time (hrs)': 'Total Transportation Time (hrs)'}, axis=1, inplace=True)
summary_large['Vehicle number'] = summary_large['Vehicle number'].astype('int64')

summary_large['Loaded quantity'] *= summary_large['Number of trips']
summary_large['Total CO2 Emission'] *= (2*summary_large['Number of trips'])
summary_large['Total Transportation Cost'] *= (2*summary_large['Number of trips'])
summary_large['Total Transportation Time (hrs)'] *= (2*summary_large['Number of trips']-1)

# result_df = summary_large.groupby(['Source', 'Destination', 'Transportation mode', 'Vehicle type']).sum().reset_index()
result_df = summary_large.groupby(['Source', 'Destination']).agg({
    'Vehicle type': 'count',
    'Total CO2 Emission': 'sum',
    'Total Transportation Time (hrs)': 'max',
    'Total Transportation Cost': 'sum',
    'Loaded quantity':'sum',
   
}).reset_index()
output_short_merged = result_df.merge(input_df, left_on=['Source', 'Destination'], right_on=['From', 'To'])
summary_short = output_short_merged[['Source', 'Destination', 'Quantity (MT)', 'Time (hrs)', 'Cost', 'Loaded quantity', 'Total Transportation Time (hrs)', 'Total Transportation Cost', 'Total CO2 Emission']]
summary_short.rename({'Quantity(MT)': 'Demand', 'Time (hrs)': 'Time Constraint (hrs)', 'Cost': 'Cost Constraint', 'Total transportation time': 'Total transportation time (hrs)', 'Loaded quantity': 'Total Quantity Transported'}, axis=1, inplace=True)    

with pd.ExcelWriter("../data/Output_data/v4/ans1/Ans_Iteration_1.xlsx") as writer:
    summary_large.to_excel(writer, sheet_name="Summary", index=False, startrow=0, startcol=0)
    summary_short.to_excel(writer, sheet_name="Summary", index=False, startrow=summary_large.shape[0]+3, startcol=0)

summary_large
# result_df
# summary_short

,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Number of trips,Loaded quantity,Total CO2 Emission,Total Transportation Cost,Total Transportation Time (hrs)
0,Delhi,Chennai,Road,4WD_PICK_UP,1,1.0,18.0,1363.023055,5255.6880,52.139762
1,Delhi,Kolkata,Rail,GOODS_RAIL,0,1.0,8000.0,531.336164,10891.8400,27.229600
2,Delhi,Kolkata,Rail,MIXED_RAIL,0,1.0,5000.0,583.206792,14976.2800,15.127556
3,Delhi,Kolkata,Road,4WD_PICK_UP,0,1.0,18.0,1018.711706,3928.0560,38.968810
4,Delhi,Kolkata,Road,HGV_(Large),0,1.0,36.0,1017.357296,4026.2574,48.137941
5,Delhi,Kolkata,Road,HGV_(Large),1,1.0,36.0,1017.357296,4026.2574,48.137941
6,Delhi,Kolkata,Road,TIPPER,0,1.0,11.0,577.299570,4451.7968,40.917250
7,Delhi,Mumbai,Rail,GOODS_RAIL,0,1.0,8000.0,464.523026,9522.2400,23.805600
8,Delhi,Mumbai,Rail,MIXED_RAIL,0,1.0,5000.0,509.871155,13093.0800,13.225333


In [44]:
%%time
###############################################
#------------------- Solve for second iteration -------------------#
###############################################
first_best_co2_emission = pulp.value(model.objective)
model += (model.objective >= first_best_co2_emission+0.00001)

#-----cost constraint------
total_cost = summary_short['Total Transportation Cost'].sum()
total_cost=total_cost-1
# model += cost_1 <= total_cost
cu=total_cost
print(total_cost)
col=[]
for s, d, m, v, c in choice_decision.keys():

    col.append(choice_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
    col.append( trip_decision[(s, d, m, v, c)] * df1.filter(pl.col("decisions") == f"{s}, {d}, {m}, {v}")["Cost"].item())
    

model += pulp.lpSum(col) <= total_cost

model.solve()
print(pulp.LpStatus[model.status])

print( first_best_co2_emission,pulp.value(model.objective))

70530.5674
Optimal
1831.2534031830812 1832.8794838311633
CPU times: total: 141 ms
Wall time: 473 ms


In [37]:
#######################################################################
# ------------------- Mappings Necessary for Output ------------------#
#######################################################################

# Extracting necessary output info. from the solver
mt_choice = []
mt_trip = []
for v in model.variables():
    if v.varValue != 0:
        if v.name.split("__(")[0] == "Choice_Decision":
            mt_choice.append(
                v.name.split("__(")[1].rstrip(")").split(",_")
                + [
                    v.varValue,
                ]
            )
        else:
            mt_trip.append(
                v.name.split("__(")[1].rstrip(")").split(",_")
                + [
                    v.varValue,
                ]
            )

mt_choice_output = pd.DataFrame(
    mt_choice,
    columns=[
        "Source",
        "Destination",
        "Transportation mode",
        "Vehicle type",
        "Vehicle number",
        "Count",
    ],
)
mt_choice_output_df = mt_choice_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_choice_output_df["S_D_M"] = (
    mt_choice_output_df["Source"] + "_" + mt_choice_output_df["Destination"] + "_" + mt_choice_output_df["Transportation mode"]
)

mt_trip_output = pd.DataFrame(
    mt_trip,
    columns=[
        "Source",
        "Destination",
        "Transportation mode",
        "Vehicle type",
        "Vehicle number",
        "Count",
    ],
)
mt_trip_output_df = mt_trip_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_trip_output_df["S_D_M"] = mt_trip_output_df["Source"] + "_" + mt_trip_output_df["Destination"] + "_" + mt_trip_output_df["Transportation mode"]
mt_trip_output_df["Vehicle type with number"] = mt_trip_output_df["Vehicle type"] + "_" + mt_trip_output_df["Vehicle number"]

# Apply all mapping to the output
dist_list = []
vals = []
for c in distances_df.columns[2:]:
    for i in range(len(distances_df)):
        dist_list.append(distances_df["Source"][i] + "_" + distances_df["Destination"][i] + "_" + c.split(" ")[0])
        vals.append(distances_df[c][i])

distance_mappings = dict(zip(dist_list, vals))
quantity_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Capacity (metric tons)"]))
emission_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Co2e (g/km)"]))
cost_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Cost per km"]))
time_mappings = dict(
    zip(
        (df["Source"] + "_" + df["Destination"] + "_" + df["Transportation modes"] + "_" + df["Vehicle_type"].str.replace(" ", "_")),
        df["Time (hrs)"],
    )
)

# Get output data ready before putting into presentable format
mt_choice_output_df["Loaded quantity"] = mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(quantity_mappings)
mt_choice_output_df["Distance"] = mt_choice_output_df["S_D_M"].map(distance_mappings)
mt_choice_output_df["Emission"] = (2 * 
    (mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(emission_mappings)) * mt_choice_output_df["Distance"] / 1000
)
mt_choice_output_df["Transportation cost"] = (mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(cost_mappings)) * mt_choice_output_df[
    "Distance"
]
mt_choice_output_df["Time (hrs)"] = (mt_choice_output_df["S_D_M"] + "_" + mt_choice_output_df["Vehicle type"]).map(time_mappings)

mt_trip_output_df["Loaded quantity"] = mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(quantity_mappings)
mt_trip_output_df["Distance"] = mt_trip_output_df["S_D_M"].map(distance_mappings)
mt_trip_output_df["Emission"] = (2 * 
    (mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(emission_mappings)) * mt_trip_output_df["Distance"] / 1000
)
mt_trip_output_df["Transportation cost"] = (mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(cost_mappings)) * mt_trip_output_df[
    "Distance"
]
mt_trip_output_df["Time (hrs)"] = (mt_trip_output_df["S_D_M"] + "_" + mt_trip_output_df["Vehicle type"]).map(time_mappings)
mt_trip_output_df["Time (hrs)"] *= mt_trip_output_df["Count"]

###########################################################
# ------------------- Generating Output ------------------#
###########################################################

# Pivot tables for more presentable data

vehicle_count_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Count",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
vehicle_count_choice["Total Vehicles in Use"] = vehicle_count_choice.iloc[:, 2:].sum(axis=1)

material_transport_quantity_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Loaded quantity",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
material_transport_quantity_choice["Total Quantity Transported"] = material_transport_quantity_choice.iloc[:, 2:].sum(axis=1)

co2_emission_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Emission",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
co2_emission_choice["Total co2 Emission (kg)"] = co2_emission_choice.iloc[:, 2:].sum(axis=1)

transportation_cost_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Transportation cost",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
transportation_cost_choice["Total transportation cost"] = transportation_cost_choice.iloc[:, 2:].sum(axis=1)

transportation_time_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Time (hrs)",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="max",
).reset_index()
transportation_time_choice["Total Time (hrs)"] = transportation_time_choice.iloc[:, 2:].sum(axis=1)

###########################

vehicle_count_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Count",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
vehicle_count_trip["Total Vehicles in Use"] = vehicle_count_trip.iloc[:, 2:].sum(axis=1)

material_transport_quantity_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Loaded quantity",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
material_transport_quantity_trip["Total Quantity Transported"] = material_transport_quantity_trip.iloc[:, 2:].sum(axis=1)

co2_emission_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Emission",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
co2_emission_trip["Total co2 Emission (kg)"] = co2_emission_trip.iloc[:, 2:].sum(axis=1)

transportation_cost_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Transportation cost",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
transportation_cost_trip["Total transportation cost"] = transportation_cost_trip.iloc[:, 2:].sum(axis=1)

transportation_time_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Time (hrs)",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="max",
).reset_index()
transportation_time_trip[" Total Time (hrs)"] = transportation_time_trip.iloc[:, 2:].sum(axis=1)

# Some cleaning

vehicle_count_choice.fillna(0, inplace=True)
material_transport_quantity_choice.fillna(0, inplace=True)
co2_emission_choice.fillna(0, inplace=True)
transportation_cost_choice.fillna(0, inplace=True)
transportation_time_choice.fillna(0, inplace=True)

vehicle_count_choice_list.append(vehicle_count_choice)
material_transport_quantity_choice_list.append(material_transport_quantity_choice)
co2_emission_choice_list.append(co2_emission_choice)
transportation_cost_choice_list.append(transportation_cost_choice)
transportation_time_choice_list.append(transportation_time_choice)

vehicle_count_trip.fillna(0, inplace=True)
material_transport_quantity_trip.fillna(0, inplace=True)
co2_emission_trip.fillna(0, inplace=True)
transportation_cost_trip.fillna(0, inplace=True)
transportation_time_trip.fillna(0, inplace=True)

vehicle_count_trip_list.append(vehicle_count_trip)
material_transport_quantity_trip_list.append(material_transport_quantity_trip)
co2_emission_trip_list.append(co2_emission_trip)
transportation_cost_trip_list.append(transportation_cost_trip)
transportation_time_trip_list.append(transportation_time_trip)

In [38]:
################################################################################
# ------------------- Generating separate output excel files ------------------#
################################################################################

with pd.ExcelWriter("../data/Output_data/v4/ans2/Vehicle_Count.xlsx") as writer1:
    vehicle_count_choice_list[0].to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    vehicle_count_trip_list[0].to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=vehicle_count_choice_list[0].shape[1] + 2)

with pd.ExcelWriter("../data/Output_data/v4/ans2/Material_Transport_Quantity.xlsx") as writer2:
    material_transport_quantity_choice_list[0].to_excel(writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    material_transport_quantity_trip_list[0].to_excel(
        writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=material_transport_quantity_choice_list[0].shape[1] + 2
    )

with pd.ExcelWriter("../data/Output_data/v4/ans2/CO2_Emission.xlsx") as writer3:
    co2_emission_choice_list[0].to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    co2_emission_trip_list[0].to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=co2_emission_choice_list[0].shape[1] + 2)

with pd.ExcelWriter("../data/Output_data/v4/ans2/Transportation_Cost.xlsx") as writer4:
    transportation_cost_choice_list[0].to_excel(writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    transportation_cost_trip_list[0].to_excel(
        writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_cost_choice_list[0].shape[1] + 2
    )

with pd.ExcelWriter("../data/Output_data/v4/ans2/Transportation_Time.xlsx") as writer5:
    transportation_time_choice_list[0].to_excel(writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    transportation_time_trip_list[0].to_excel(
        writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_time_choice_list[0].shape[1] + 2
    )

In [39]:
##################################################################
# ------------------- Final Summary and Result ------------------#
##################################################################

output_large_merged = mt_choice_output_df.merge(mt_trip_output_df[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count', 'Time (hrs)']], on=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number'], how="left").fillna(-1)
output_large_merged['Count_y']+=1
output_large_merged['Count_y'] = output_large_merged[['Count_x', 'Count_y']].max(axis=1)
output_large_merged['Time (hrs)'] = output_large_merged[['Time (hrs)_x', 'Time (hrs)_y']].max(axis=1)
summary_large = output_large_merged[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count_y', 'Loaded quantity', 'Emission', 'Transportation cost', 'Time (hrs)']]
summary_large.rename({'Count_y': 'Number of trips', 'Emission': 'Total CO2 Emission', 'Transportation cost': 'Total Transportation Cost', 'Time (hrs)': 'Total Transportation Time (hrs)'}, axis=1, inplace=True)
summary_large['Vehicle number'] = summary_large['Vehicle number'].astype('int64')

summary_large['Loaded quantity'] *= summary_large['Number of trips']
summary_large['Total CO2 Emission'] *= (2*summary_large['Number of trips'])
summary_large['Total Transportation Cost'] *= (2*summary_large['Number of trips'])
summary_large['Total Transportation Time (hrs)'] *= (2*summary_large['Number of trips']-1)

# result_df = summary_large.groupby(['Source', 'Destination', 'Transportation mode', 'Vehicle type']).sum().reset_index()
result_df = summary_large.groupby(['Source', 'Destination']).agg({
     'Vehicle type': 'count',
    'Total CO2 Emission': 'sum',
    'Total Transportation Time (hrs)': 'max',
    'Total Transportation Cost': 'sum',
    'Loaded quantity':'sum',
   
}).reset_index()
output_short_merged = result_df.merge(input_df, left_on=['Source', 'Destination'], right_on=['From', 'To'])
summary_short = output_short_merged[['Source', 'Destination', 'Quantity (MT)', 'Time (hrs)', 'Cost', 'Loaded quantity', 'Total Transportation Time (hrs)', 'Total Transportation Cost', 'Total CO2 Emission']]
summary_short.rename({'Quantity(MT)': 'Demand', 'Time (hrs)': 'Time Constraint (hrs)', 'Cost': 'Cost Constraint', 'Total transportation time': 'Total transportation time (hrs)', 'Loaded quantity': 'Total Quantity Transported'}, axis=1, inplace=True)    

with pd.ExcelWriter("../data/Output_data/v4/ans2/Ans_Iteration_2.xlsx") as writer:
    summary_large.to_excel(writer, sheet_name="Summary", index=False, startrow=0, startcol=0)
    summary_short.to_excel(writer, sheet_name="Summary", index=False, startrow=summary_large.shape[0]+3, startcol=0)

summary_large

,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Number of trips,Loaded quantity,Total CO2 Emission,Total Transportation Cost,Total Transportation Time (hrs)
0,Delhi,Chennai,Road,4WD_PICK_UP,1,1.0,18.0,1363.023055,5255.6880,52.139762
1,Delhi,Kolkata,Rail,GOODS_RAIL,0,1.0,8000.0,531.336164,10891.8400,27.229600
2,Delhi,Kolkata,Rail,MIXED_RAIL,0,1.0,5000.0,583.206792,14976.2800,15.127556
3,Delhi,Kolkata,Road,HGV_(Large),0,1.0,36.0,1017.357296,4026.2574,48.137941
4,Delhi,Kolkata,Road,HGV_(Large),1,1.0,36.0,1017.357296,4026.2574,48.137941
5,Delhi,Kolkata,Road,HGV_DROPSIDE_LORRY,1,1.0,24.0,1025.216029,4255.3940,36.370889
6,Delhi,Kolkata,Road,TIPPER,0,1.0,11.0,577.299570,4451.7968,40.917250
7,Delhi,Mumbai,Rail,GOODS_RAIL,0,1.0,8000.0,464.523026,9522.2400,23.805600
8,Delhi,Mumbai,Rail,MIXED_RAIL,0,1.0,5000.0,509.871155,13093.0800,13.225333


In [40]:
%%time
###################################################################
#------------------- Solve for third iteration -------------------#
###################################################################
second_best_co2_emission = pulp.value(model.objective)
model += (model.objective >= second_best_co2_emission+0.00001)
total_cost = summary_short['Total Transportation Cost'].sum()
total_cost=total_cost-1
model += pulp.lpSum(col) <= total_cost
print(total_cost)
model.solve()
print(pulp.LpStatus[model.status])
print(first_best_co2_emission,second_best_co2_emission,pulp.value(model.objective))

70497.83360000001
Optimal
1770.6715150786888 1772.2975957267709 1831.2534031830812
CPU times: total: 46.9 ms
Wall time: 206 ms


In [41]:
#######################################################################
# ------------------- Mappings Necessary for Output ------------------#
#######################################################################

# Extracting necessary output info. from the solver
mt_choice = []
mt_trip = []
for v in model.variables():
    if v.varValue != 0:
        if v.name.split("__(")[0] == "Choice_Decision":
            mt_choice.append(
                v.name.split("__(")[1].rstrip(")").split(",_")
                + [
                    v.varValue,
                ]
            )
        else:
            mt_trip.append(
                v.name.split("__(")[1].rstrip(")").split(",_")
                + [
                    v.varValue,
                ]
            )

mt_choice_output = pd.DataFrame(
    mt_choice,
    columns=[
        "Source",
        "Destination",
        "Transportation mode",
        "Vehicle type",
        "Vehicle number",
        "Count",
    ],
)
mt_choice_output_df = mt_choice_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_choice_output_df["S_D_M"] = (
    mt_choice_output_df["Source"] + "_" + mt_choice_output_df["Destination"] + "_" + mt_choice_output_df["Transportation mode"]
)

mt_trip_output = pd.DataFrame(
    mt_trip,
    columns=[
        "Source",
        "Destination",
        "Transportation mode",
        "Vehicle type",
        "Vehicle number",
        "Count",
    ],
)
mt_trip_output_df = mt_trip_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_trip_output_df["S_D_M"] = mt_trip_output_df["Source"] + "_" + mt_trip_output_df["Destination"] + "_" + mt_trip_output_df["Transportation mode"]
mt_trip_output_df["Vehicle type with number"] = mt_trip_output_df["Vehicle type"] + "_" + mt_trip_output_df["Vehicle number"]

# Apply all mapping to the output
dist_list = []
vals = []
for c in distances_df.columns[2:]:
    for i in range(len(distances_df)):
        dist_list.append(distances_df["Source"][i] + "_" + distances_df["Destination"][i] + "_" + c.split(" ")[0])
        vals.append(distances_df[c][i])

distance_mappings = dict(zip(dist_list, vals))
quantity_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Capacity (metric tons)"]))
emission_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Co2e (g/km)"]))
cost_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Cost per km"]))
time_mappings = dict(
    zip(
        (df["Source"] + "_" + df["Destination"] + "_" + df["Transportation modes"] + "_" + df["Vehicle_type"].str.replace(" ", "_")),
        df["Time (hrs)"],
    )
)

# Get output data ready before putting into presentable format
mt_choice_output_df["Loaded quantity"] = mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(quantity_mappings)
mt_choice_output_df["Distance"] = mt_choice_output_df["S_D_M"].map(distance_mappings)
mt_choice_output_df["Emission"] = (2 * 
    (mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(emission_mappings)) * mt_choice_output_df["Distance"] / 1000
)
mt_choice_output_df["Transportation cost"] = (mt_choice_output_df["Vehicle type"].str.replace("_", " ").map(cost_mappings)) * mt_choice_output_df[
    "Distance"
]
mt_choice_output_df["Time (hrs)"] = (mt_choice_output_df["S_D_M"] + "_" + mt_choice_output_df["Vehicle type"]).map(time_mappings)

mt_trip_output_df["Loaded quantity"] = mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(quantity_mappings)
mt_trip_output_df["Distance"] = mt_trip_output_df["S_D_M"].map(distance_mappings)
mt_trip_output_df["Emission"] = (2 * 
    (mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(emission_mappings)) * mt_trip_output_df["Distance"] / 1000
)
mt_trip_output_df["Transportation cost"] = (mt_trip_output_df["Vehicle type"].str.replace("_", " ").map(cost_mappings)) * mt_trip_output_df[
    "Distance"
]
mt_trip_output_df["Time (hrs)"] = (mt_trip_output_df["S_D_M"] + "_" + mt_trip_output_df["Vehicle type"]).map(time_mappings)
mt_trip_output_df["Time (hrs)"] *= mt_trip_output_df["Count"]

###########################################################
# ------------------- Generating Output ------------------#
###########################################################

# Pivot tables for more presentable data

vehicle_count_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Count",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
vehicle_count_choice["Total Vehicles in Use"] = vehicle_count_choice.iloc[:, 2:].sum(axis=1)

material_transport_quantity_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Loaded quantity",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
material_transport_quantity_choice["Total Quantity Transported"] = material_transport_quantity_choice.iloc[:, 2:].sum(axis=1)

co2_emission_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Emission",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
co2_emission_choice["Total co2 Emission (kg)"] = co2_emission_choice.iloc[:, 2:].sum(axis=1)

transportation_cost_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Transportation cost",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="sum",
).reset_index()
transportation_cost_choice["Total transportation cost"] = transportation_cost_choice.iloc[:, 2:].sum(axis=1)

transportation_time_choice = pd.pivot_table(
    mt_choice_output_df,
    values="Time (hrs)",
    index=["Source", "Destination"],
    columns=["Vehicle type"],
    aggfunc="max",
).reset_index()
transportation_time_choice["Total Time (hrs)"] = transportation_time_choice.iloc[:, 2:].sum(axis=1)

###########################

vehicle_count_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Count",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
vehicle_count_trip["Total Vehicles in Use"] = vehicle_count_trip.iloc[:, 2:].sum(axis=1)

material_transport_quantity_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Loaded quantity",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
material_transport_quantity_trip["Total Quantity Transported"] = material_transport_quantity_trip.iloc[:, 2:].sum(axis=1)

co2_emission_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Emission",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
co2_emission_trip["Total co2 Emission (kg)"] = co2_emission_trip.iloc[:, 2:].sum(axis=1)

transportation_cost_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Transportation cost",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="sum",
).reset_index()
transportation_cost_trip["Total transportation cost"] = transportation_cost_trip.iloc[:, 2:].sum(axis=1)

transportation_time_trip = pd.pivot_table(
    mt_trip_output_df,
    values="Time (hrs)",
    index=["Source", "Destination"],
    columns=["Vehicle type with number"],
    aggfunc="max",
).reset_index()
transportation_time_trip[" Total Time (hrs)"] = transportation_time_trip.iloc[:, 2:].sum(axis=1)

# Some cleaning

vehicle_count_choice.fillna(0, inplace=True)
material_transport_quantity_choice.fillna(0, inplace=True)
co2_emission_choice.fillna(0, inplace=True)
transportation_cost_choice.fillna(0, inplace=True)
transportation_time_choice.fillna(0, inplace=True)

vehicle_count_choice_list.append(vehicle_count_choice)
material_transport_quantity_choice_list.append(material_transport_quantity_choice)
co2_emission_choice_list.append(co2_emission_choice)
transportation_cost_choice_list.append(transportation_cost_choice)
transportation_time_choice_list.append(transportation_time_choice)

vehicle_count_trip.fillna(0, inplace=True)
material_transport_quantity_trip.fillna(0, inplace=True)
co2_emission_trip.fillna(0, inplace=True)
transportation_cost_trip.fillna(0, inplace=True)
transportation_time_trip.fillna(0, inplace=True)

vehicle_count_trip_list.append(vehicle_count_trip)
material_transport_quantity_trip_list.append(material_transport_quantity_trip)
co2_emission_trip_list.append(co2_emission_trip)
transportation_cost_trip_list.append(transportation_cost_trip)
transportation_time_trip_list.append(transportation_time_trip)

In [42]:
################################################################################
# ------------------- Generating separate output excel files ------------------#
################################################################################

with pd.ExcelWriter("../data/Output_data/v4/ans3/Vehicle_Count.xlsx") as writer1:
    vehicle_count_choice_list[0].to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    vehicle_count_trip_list[0].to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=vehicle_count_choice_list[0].shape[1] + 2)

with pd.ExcelWriter("../data/Output_data/v4/ans3/Material_Transport_Quantity.xlsx") as writer2:
    material_transport_quantity_choice_list[0].to_excel(writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    material_transport_quantity_trip_list[0].to_excel(
        writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=material_transport_quantity_choice_list[0].shape[1] + 2
    )

with pd.ExcelWriter("../data/Output_data/v4/ans3/CO2_Emission.xlsx") as writer3:
    co2_emission_choice_list[0].to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    co2_emission_trip_list[0].to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=co2_emission_choice_list[0].shape[1] + 2)

with pd.ExcelWriter("../data/Output_data/v4/ans3/Transportation_Cost.xlsx") as writer4:
    transportation_cost_choice_list[0].to_excel(writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    transportation_cost_trip_list[0].to_excel(
        writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_cost_choice_list[0].shape[1] + 2
    )

with pd.ExcelWriter("../data/Output_data/v4/ans3/Transportation_Time.xlsx") as writer5:
    transportation_time_choice_list[0].to_excel(writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
    transportation_time_trip_list[0].to_excel(
        writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_time_choice_list[0].shape[1] + 2
    )

In [43]:
##################################################################
# ------------------- Final Summary and Result ------------------#
##################################################################

output_large_merged = mt_choice_output_df.merge(mt_trip_output_df[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count', 'Time (hrs)']], on=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number'], how="left").fillna(-1)
output_large_merged['Count_y']+=1
output_large_merged['Count_y'] = output_large_merged[['Count_x', 'Count_y']].max(axis=1)
output_large_merged['Time (hrs)'] = output_large_merged[['Time (hrs)_x', 'Time (hrs)_y']].max(axis=1)
summary_large = output_large_merged[['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count_y', 'Loaded quantity', 'Emission', 'Transportation cost', 'Time (hrs)']]
summary_large.rename({'Count_y': 'Number of trips', 'Emission': 'Total CO2 Emission', 'Transportation cost': 'Total Transportation Cost', 'Time (hrs)': 'Total Transportation Time (hrs)'}, axis=1, inplace=True)
summary_large['Vehicle number'] = summary_large['Vehicle number'].astype('int64')

summary_large['Loaded quantity'] *= summary_large['Number of trips']
summary_large['Total CO2 Emission'] *= (2*summary_large['Number of trips'])
summary_large['Total Transportation Cost'] *= (2*summary_large['Number of trips'])
summary_large['Total Transportation Time (hrs)'] *= (2*summary_large['Number of trips']-1)

# result_df = summary_large.groupby(['Source', 'Destination', 'Transportation mode', 'Vehicle type']).sum().reset_index()
result_df = summary_large.groupby(['Source', 'Destination']).agg({
     'Vehicle type': 'count',
    'Total CO2 Emission': 'sum',
    'Total Transportation Time (hrs)': 'max',
    'Total Transportation Cost': 'sum',
    'Loaded quantity':'sum',
   
}).reset_index()
output_short_merged = result_df.merge(input_df, left_on=['Source', 'Destination'], right_on=['From', 'To'])
summary_short = output_short_merged[['Source', 'Destination', 'Quantity (MT)', 'Time (hrs)', 'Cost', 'Loaded quantity', 'Total Transportation Time (hrs)', 'Total Transportation Cost', 'Total CO2 Emission']]
summary_short.rename({'Quantity(MT)': 'Demand', 'Time (hrs)': 'Time Constraint (hrs)', 'Cost': 'Cost Constraint', 'Total transportation time': 'Total transportation time (hrs)', 'Loaded quantity': 'Total Quantity Transported'}, axis=1, inplace=True)    

with pd.ExcelWriter("../data/Output_data/v4/ans3/Ans_Iteration_3.xlsx") as writer:
    summary_large.to_excel(writer, sheet_name="Summary", index=False, startrow=0, startcol=0)
    summary_short.to_excel(writer, sheet_name="Summary", index=False, startrow=summary_large.shape[0]+3, startcol=0)

summary_large

,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Number of trips,Loaded quantity,Total CO2 Emission,Total Transportation Cost,Total Transportation Time (hrs)
0,Delhi,Chennai,Road,4WD_PICK_UP,1,1.0,18.0,1363.023055,5255.6880,52.139762
1,Delhi,Kolkata,Rail,GOODS_RAIL,0,1.0,8000.0,531.336164,10891.8400,27.229600
2,Delhi,Kolkata,Rail,MIXED_RAIL,0,1.0,5000.0,583.206792,14976.2800,15.127556
3,Delhi,Kolkata,Road,4WD_PICK_UP,0,1.0,18.0,1018.711706,3928.0560,38.968810
4,Delhi,Kolkata,Road,HGV_(Large),0,1.0,36.0,1017.357296,4026.2574,48.137941
5,Delhi,Kolkata,Road,HGV_(Large),1,1.0,36.0,1017.357296,4026.2574,48.137941
6,Delhi,Kolkata,Road,MOTOR_CARAVAN,1,1.0,13.0,819.627123,4811.8686,32.733800
7,Delhi,Mumbai,Rail,GOODS_RAIL,0,1.0,8000.0,464.523026,9522.2400,23.805600
8,Delhi,Mumbai,Rail,MIXED_RAIL,0,1.0,5000.0,509.871155,13093.0800,13.225333
